In [4]:
import tensorflow as tf
from tensorflow.contrib import slim
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import os
from PIL import Image
from collections import OrderedDict
import torchvision.transforms as transforms
from old_generator import TorchGAN

%load_ext autoreload
%autoreload 2

In [5]:
# os.system("gdown --id 1TE1Bi5ym-TbLSnlkAAOP0sSQYkgTNl4A --output datasets/omniglot_data.npy")
# os.system("mkdir checkpoints")
# os.system("gdown --id 1qw_op6L2RebrGik0cBWquDK3RTculrVR --output checkpoints/model.ckpt.data-00000-of-00001")
# os.system("gdown --id 17UFRFwWOyJ_-tU72SH0o5mrg2-oEra95 --output checkpoints/model.ckpt.index")
# os.system("gdown --id 1iSOAjMUWLDemHljNHk0grCFdblSArBcP --output checkpoints/model.ckpt.meta")

In [6]:
raw_data = np.load("datasets/omniglot_data.npy")

In [7]:
curr_data = raw_data[1200]

In [8]:
from dagan_architectures import UResNetGenerator, Discriminator

d = Discriminator(batch_size=1, layer_sizes=[64, 64, 128, 128],
                        inner_layers=[5] * 4, name="discriminator")

In [9]:
inp1 = tf.placeholder(tf.float32, [1,28,28,1], 'inp1')
inp2 = tf.placeholder(tf.float32, [1,28,28,1], 'inp2')

In [10]:
result = d(inp1, inp2, training=False)


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


discr layers 22
discriminator_parameter_num 7072551


In [11]:
simple_inp = np.ones((1, 28, 28, 1))

In [12]:
checkpoint = "checkpoints/checkpoint.ckpt"
ckpt_reader = tf.train.load_checkpoint(checkpoint)
def read_tf(var):
    return torch.tensor(ckpt_reader.get_tensor(var))

def reshape_conv(tensor):
    return tensor.transpose(0, 3).transpose(1, 2).transpose(2, 3)

In [13]:
# Run tf model

fine_tune = slim.assign_from_checkpoint_fn(
    checkpoint,
    tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

sess = tf.Session()
fine_tune(sess)

final_results = sess.run(
    result,
    feed_dict={inp1: simple_inp, inp2: simple_inp},
)
np.array(final_results[0])

INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt


array([[-170.03334]], dtype=float32)

In [40]:
final_results[1][-1][0,:,:,0]

array([[2.7638643, 1.9944654],
       [1.5974588, 0.5871061]], dtype=float32)

In [15]:
final_results[2][3][0, 0, :, 0]

array([-3.2612257, -3.2612257, -3.2612257, -3.2612257, -3.2612257,
       -3.2612257, -3.2612257, -3.2612257, -3.2612257, -3.2612257,
       -3.2612257, -3.2612257, -3.2612257, -3.270965 ], dtype=float32)

In [28]:
from collections import OrderedDict

import torch
from torch import nn
import torch.nn.functional as F


class _LayerNorm(nn.Module):
    def __init__(self, num_features, img_size):
        """
        Normalizes over the entire image and scales + weights for each feature
        """
        super().__init__()
        self.layer_norm = nn.LayerNorm((num_features, img_size, img_size), elementwise_affine=False, eps=1e-12)
        self.weight = torch.nn.Parameter(torch.ones(num_features).float().unsqueeze(-1).unsqueeze(-1), requires_grad=True)
        self.bias = torch.nn.Parameter(torch.zeros(num_features).float().unsqueeze(-1).unsqueeze(-1), requires_grad=True)

    def forward(self, x):
        out = self.layer_norm(x)
        out = out * self.weight + self.bias
        return out

class _SamePad(nn.Module):
    """
    Pads equivalent to the behavior of tensorflow "SAME"
    """
    def __init__(self, stride):
        super().__init__()
        self.stride = stride
    
    def forward(self, x):
        if self.stride == 2 and x.shape[2] % 2 == 0:
            return F.pad(x, (0, 1, 0, 1))
        return F.pad(x, (1, 1, 1, 1))

def _conv2d(in_channels, out_channels, kernel_size, stride, out_size=None, activate=True, dropout=0.0):
    layers = OrderedDict()
    layers['pad'] = _SamePad(stride)
    layers['conv'] = nn.Conv2d(in_channels, out_channels, kernel_size, stride)
    if activate:
        if out_size is None:
            raise ValueError("Must provide out_size if activate is True")
        layers['relu'] = nn.LeakyReLU(0.2)
        layers['norm'] = _LayerNorm(out_channels, out_size)
    
    if dropout > 0.0:
        layers['dropout'] = nn.Dropout(dropout)
    return nn.Sequential(layers)


class _EncoderBlock(nn.Module):
    def __init__(
        self, pre_channels, in_channels, out_channels, num_layers, out_size, dropout_rate=0.0
    ):
        super().__init__()
        self.num_layers = num_layers
        self.pre_conv = _conv2d(
            in_channels=pre_channels, out_channels=pre_channels, kernel_size=3, stride=2, activate=False
        )

        self.conv0 = _conv2d(
            in_channels=in_channels + pre_channels, out_channels=out_channels, kernel_size=3, stride=1, out_size=out_size
        )
        total_channels = in_channels + out_channels
        for i in range(1, num_layers):
            self.add_module(
                'conv%d' % i,
                _conv2d(
                    in_channels=total_channels, out_channels=out_channels, kernel_size=3, stride=1, out_size=out_size
                )
            )
            total_channels += out_channels
        self.add_module(
            'conv%d' % num_layers,
            _conv2d(
                in_channels=total_channels, out_channels=out_channels, kernel_size=3, stride=2, out_size=(out_size + 1) // 2, dropout=dropout_rate
            )
        )

    def forward(self, inp):
        pre_input, x = inp
        pre_input = self.pre_conv(pre_input)
        out = self.conv0(torch.cat([x, pre_input], 1))
        
        all_outputs = [x, out]
        for i in range(1, self.num_layers + 1):
            input_features = torch.cat([all_outputs[-1], all_outputs[-2]] + all_outputs[:-2], 1)
            module = self._modules['conv%d' % i]
            out = module(input_features)
            all_outputs.append(out)
        return all_outputs[-2], all_outputs[-1]


class TorchDiscriminator(nn.Module):
    def __init__(self, dim, channels, dropout_rate=0.0, z_dim=100):
        super().__init__()
        self.dim = dim
        self.z_dim = z_dim
        self.channels = channels
        self.layer_sizes = [64, 64, 128, 128]
        self.num_inner_layers = 5

        # Number of times dimension is halved
        self.depth = len(self.layer_sizes)
        
        # dimension at each level of U-net
        self.dim_arr = [dim]
        for i in range(self.depth):
            self.dim_arr.append((self.dim_arr[-1] + 1) // 2)
        
        # Encoders
        self.encode0 = _conv2d(
            in_channels=self.channels * 2, out_channels=self.layer_sizes[0],
            kernel_size=3, stride=2, out_size=self.dim_arr[1]
        )
        for i in range(1, self.depth):
            self.add_module(
                "encode%d" % i,
                _EncoderBlock(
                    pre_channels=self.channels if i == 1 else self.layer_sizes[i - 1],
                    in_channels=self.layer_sizes[i - 1],
                    out_channels=self.layer_sizes[i],
                    num_layers=self.num_inner_layers,
                    out_size=self.dim_arr[i],
                    dropout_rate=dropout_rate,
                )
            )
        
#         # Noise encoders
#         self.noise_encoders = 3
#         num_noise_filters = 8
#         self.z_channels = []
#         for i in range(self.noise_encoders):
#             curr_dim = self.dim_arr[-1 - i]  # Iterate dim from back
#             self.add_module(
#                 "z_reshape%d" % i,
#                 nn.Linear(self.z_dim, curr_dim * curr_dim * num_noise_filters)
#             )
#             self.z_channels.append(num_noise_filters)
#             num_noise_filters //= 2
            
#         # Decoders
#         for i in range(self.U_depth + 1):
#             # Input from previous decoder
#             in_channels = 0 if i == 0 else self.layer_sizes[-i]
#             # Input from encoder across the "U"
#             in_channels += self.channels if i == self.U_depth else self.layer_sizes[-i - 1]
#             # Input from injected noise
#             if i < self.noise_encoders:
#                 in_channels += self.z_channels[i]

#             self.add_module(
#                 "decode%d" % i,
#                 _DecoderBlock(
#                     pre_channels=0 if i == 0 else self.layer_sizes[-i],
#                     in_channels=in_channels,
#                     out_channels=self.layer_sizes[0] if i == self.U_depth else self.layer_sizes[-i - 1],
#                     num_layers=self.num_inner_layers,
#                     curr_size=self.dim_arr[-i - 1],
#                     upscale_size=None if i == self.U_depth else self.dim_arr[-i - 2],
#                     dropout_rate=dropout_rate,
#                 )
#             )

#         # Final conv
#         self.num_final_conv = 3
#         for i in range(self.num_final_conv - 1):
#             self.add_module(
#                 "final_conv%d" % i,
#                 _conv2d(
#                     in_channels=self.layer_sizes[0], out_channels=self.layer_sizes[0], kernel_size=3, stride=1
#                 )
#             )
#         self.add_module(
#             "final_conv%d" % (self.num_final_conv - 1),
#             _conv2d(
#                 in_channels=self.layer_sizes[0], out_channels=self.channels, kernel_size=3, stride=1, activate=False
#             )
#         )
#         self.tanh = nn.Tanh()

    def forward(self, x1, x2):
        x = torch.cat([x1, x2], 1)
        out = [x, self.encode0(x)]
        for i in range(1, len(self.layer_sizes)):
            out = self._modules["encode%d" % i](out)
        return out[1]

In [29]:
torch_d = TorchDiscriminator(28, 1)

In [33]:
# Seed torch generator weights

def convert_conv_layer(torch_layer, conv_path, norm_path=None):
    if norm_path is not None:
        torch_layer.norm.weight.data = read_tf(norm_path + "/gamma").unsqueeze(-1).unsqueeze(-1)
        torch_layer.norm.bias.data = read_tf(norm_path + "/beta").unsqueeze(-1).unsqueeze(-1)
    
    torch_layer.conv.weight.data = reshape_conv(read_tf(conv_path + "/kernel"))
    torch_layer.conv.bias.data = read_tf(conv_path + "/bias")
    
num_layers = torch_d.num_inner_layers

# Encoders
convert_conv_layer(
    torch_d.encode0, "discriminator/conv_layers/g_conv0/conv2d", "discriminator/conv_layers/g_conv0/LayerNorm")

for i in range(1, torch_d.depth):
    module = torch_d._modules["encode%d" % i]
    convert_conv_layer(
        module.pre_conv,
        f"discriminator/conv_layers/g_conv{i}/conv2d"
    )

    for j in range(num_layers + 1):
        norm_suffix = f"_{j}" if j > 0 else ""
        convert_conv_layer(
            module._modules[f"conv{j}"],
            f"discriminator/conv_layers/g_conv{i}/conv2d_{j + 1}",
            f"discriminator/conv_layers/g_conv{i}/LayerNorm{norm_suffix}",
        )

# # Noise encoders
# for i in range(torch_d.noise_encoders):
#     module = torch_d._modules["z_reshape%d" % i]
#     z_suffix = f"_{i}" if i > 0 else ""
#     module.weight.data = read_tf(f"generator/vector_expansion/dense{z_suffix}/kernel").T
#     module.bias.data = read_tf(f"generator/vector_expansion/dense{z_suffix}/bias").T


# # Decoders
# for i in range(torch_d.U_depth + 1):
#     module = torch_d._modules["decode%d" % i]

#     for j in range(num_layers):
#         idx_suffix = f"_{j}" if j > 0 else ""
#         if i > 0:
#             convert_conv_layer(
#                 module._modules[f"pre_conv_t{j}"],
#                 f"generator/g_deconv_layers/g_deconv{i}/conv2d_transpose{idx_suffix}",
#             )
#         convert_conv_layer(
#             module._modules[f"conv{j}"],
#             f"generator/g_deconv_layers/g_deconv{i}/conv2d{idx_suffix}",
#             f"generator/g_deconv_layers/g_deconv{i}/BatchNorm{idx_suffix}",
#         )
#     if i < torch_d.U_depth:
#         conv_suffix = f"_{num_layers}" if i > 0 else ""
#         convert_conv_layer(
#             module._modules[f"conv_t{num_layers}"],
#             f"generator/g_deconv_layers/g_deconv{i}/conv2d_transpose{conv_suffix}",
#             f"generator/g_deconv_layers/g_deconv{i}/BatchNorm_{num_layers}",
#         )
        
# # Final conv
# for i in range(torch_d.num_final_conv):
#     idx_suffix = f"_{i}" if i > 0 else ""
#     if i == torch_d.num_final_conv - 1:
#         bn_path = None
#     else:
#         bn_path = f"generator/g_deconv_layers/BatchNorm{idx_suffix}"
#     convert_conv_layer(
#         torch_d._modules[f"final_conv{i}"],
#         f"generator/g_deconv_layers/conv2d{idx_suffix}",
#         bn_path
#     )

In [34]:
def to_torch(arr):
    return torch.tensor(arr).transpose(0, 1).transpose(0, 2).unsqueeze(0)
x1 = torch.ones((1, 1, 28, 28))#to_torch(curr_data[0])
x2 = torch.ones((1, 1, 28, 28))#to_torch(curr_data[1])

In [35]:
out = torch_d(x1, x2)

In [42]:
out[0][0]

tensor([[2.7639, 1.9945],
        [1.5975, 0.5871]], grad_fn=<SelectBackward>)

In [36]:
out.shape

torch.Size([1, 128, 2, 2])

In [ ]:
out.shape

In [ ]:
def render_image_arr(arr):
    arr = np.uint8(arr * 256)
    arr = arr.reshape(arr.shape[:-1])
    display(Image.fromarray(arr, mode='L').resize((224, 224)))
    
def render_torch(arr):
    arr = np.uint8(arr * 256)
    display(Image.fromarray(arr, mode='L').resize((224, 224)))

In [ ]:
# Render tf
render_image_arr(np.array(final_results[0][0]) * 0.5 + 0.5)

In [ ]:
# Render torch
refined_torch_out = torch_out.detach().numpy() * 0.5 + 0.5

render_torch(refined_torch_out[0][0])

In [24]:
tf.train.list_variables(checkpoint)

[('beta1_power', []),
 ('beta1_power_1', []),
 ('beta2_power', []),
 ('beta2_power_1', []),
 ('discriminator/conv_layers/g_conv0/LayerNorm/beta', [64]),
 ('discriminator/conv_layers/g_conv0/LayerNorm/beta/Adam', [64]),
 ('discriminator/conv_layers/g_conv0/LayerNorm/beta/Adam_1', [64]),
 ('discriminator/conv_layers/g_conv0/LayerNorm/gamma', [64]),
 ('discriminator/conv_layers/g_conv0/LayerNorm/gamma/Adam', [64]),
 ('discriminator/conv_layers/g_conv0/LayerNorm/gamma/Adam_1', [64]),
 ('discriminator/conv_layers/g_conv0/conv2d/bias', [64]),
 ('discriminator/conv_layers/g_conv0/conv2d/bias/Adam', [64]),
 ('discriminator/conv_layers/g_conv0/conv2d/bias/Adam_1', [64]),
 ('discriminator/conv_layers/g_conv0/conv2d/kernel', [3, 3, 2, 64]),
 ('discriminator/conv_layers/g_conv0/conv2d/kernel/Adam', [3, 3, 2, 64]),
 ('discriminator/conv_layers/g_conv0/conv2d/kernel/Adam_1', [3, 3, 2, 64]),
 ('discriminator/conv_layers/g_conv1/LayerNorm/beta', [64]),
 ('discriminator/conv_layers/g_conv1/LayerNorm/be

In [ ]:
read_tf('discriminator/conv_layers/g_conv0/LayerNorm/beta')

In [ ]:
var = tf.trainable_variables()[-2]
var = tf.assign(var, [1., 2., 3.])
sess.run(var)
# var.eval(sess)

In [ ]:
tf.trainable_variables()[-2].assign([1,2,3],sess)